In [1]:
import numpy as np
from math import pi

# -------- configuration --------
N_BINS = 100          # histogram bins for residual PDFs
EPS     = 1e-16       # numerical guard
# --------------------------------


def angular_difference(pred, true):
    """Wrap angle difference into (–π, π]."""
    return (pred - true + np.pi) % (2 * np.pi) - np.pi


def separation_power(pdf_p, pdf_q):
    """Triangular discrimination (separation power) between two PDFs."""
    denom = pdf_p + pdf_q + EPS
    return 0.5 * np.sum((pdf_p - pdf_q) ** 2 / denom)


# ---------- load data ----------
data_smeft1 = np.load('SMEFTNET1.npy')
data_smeft2 = np.load('SMEFTNET2.npy')
data_bit100 = np.load('BITNET_100%.npy')
data_bit70  = np.load('BITNET_70%.npy')
data_bit30  = np.load('BITNET_30%.npy')

# ground truth (same for all files) and reference prediction
phi_true      = data_smeft1[:, 1]
phi_pred_ref  = data_smeft1[:, 0]      # SMEFTNet-1 as reference

# build reference residual histogram
bins          = np.linspace(-pi, pi, N_BINS + 1)
res_ref       = angular_difference(phi_pred_ref, phi_true)
pdf_ref, _    = np.histogram(res_ref, bins=bins, density=True)

# models to compare with reference
compare_dict = {
    "SMEFTNet-2"     : data_smeft2[:, 0],
    "Bit100"         : data_bit100[:, 0],
    "Bit70"          : data_bit70[:, 0],
    "Bit30"          : data_bit30[:, 0],
}

# compute and print separation power for each model
for label, pred in compare_dict.items():
    res_model   = angular_difference(pred, phi_true)
    pdf_model, _ = np.histogram(res_model, bins=bins, density=True)
    sep = separation_power(pdf_ref, pdf_model)
    print(f"{label} vs SMEFTNet-1 separation power: {sep:.6f}")



SMEFTNet-2 vs SMEFTNet-1 separation power: 0.000128
Bit100 vs SMEFTNet-1 separation power: 1.485956
Bit70 vs SMEFTNet-1 separation power: 0.613751
Bit30 vs SMEFTNet-1 separation power: 0.150274


In [2]:
import numpy as np
from math import pi
from scipy.stats import wasserstein_distance   # 1-Wasserstein (Earth-Mover) distance

# -------- configuration --------
def angular_difference(pred, true):
    """Wrap angle difference into (–π, π]."""
    return (pred - true + np.pi) % (2 * np.pi) - np.pi


# ---------- load data ----------
data_smeft1 = np.load('SMEFTNET1.npy')
data_smeft2 = np.load('SMEFTNET2.npy')
data_bit100 = np.load('BITNET_100%.npy')
data_bit70  = np.load('BITNET_70%.npy')
data_bit30  = np.load('BITNET_30%.npy')

# reference truth and prediction
phi_true      = data_smeft1[:, 1]
phi_pred_ref  = data_smeft1[:, 0]          # SMEFTNet-1

# residuals for reference model (wrapped)
res_ref = angular_difference(phi_pred_ref, phi_true)

# comparison models
compare_dict = {
    "SMEFTNet-2" : data_smeft2[:, 0],
    "Bit100"     : data_bit100[:, 0],
    "Bit70"      : data_bit70[:, 0],
    "Bit30"      : data_bit30[:, 0],
}

# compute and print Wasserstein-1 distance (units: radians)
for label, pred in compare_dict.items():
    res_model = angular_difference(pred, phi_true)
    wd = wasserstein_distance(res_ref, res_model)
    print(f"{label} vs SMEFTNet-1 Wasserstein distance: {wd:.6f} rad")



SMEFTNet-2 vs SMEFTNet-1 Wasserstein distance: 0.002072 rad
Bit100 vs SMEFTNet-1 Wasserstein distance: 0.454677 rad
Bit70 vs SMEFTNet-1 Wasserstein distance: 0.252825 rad
Bit30 vs SMEFTNet-1 Wasserstein distance: 0.104013 rad
